In [1]:
from flask import Flask, render_template, url_for, request, send_file , make_response
import pandas as pd
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from os import path, getcwd
import matplotlib.pyplot as plt
import os
import csv 


app = Flask(__name__) 
    

@app.route('/data', methods = ['POST', 'GET'])


def data():
    
    #extract the data from the request
    data = request.get_json()
    doc = data["document"]
    min_freq = data["minimum_frequency"]
    max_words = data["maximum_number_words"]
    Title = data['Title']
  

    path = os.path.dirname(doc)  #this extracts the path for later use. Not sure if it works on apple
    
    df = pd.read_csv(doc)  #read the file
    
    
    #clean the data
    df = df.dropna()
    
    df_1 = df.loc[df['Title'] == str(Title)]
    
    l = list(df_1['reviews_submitted'].unique())
    
    text = " ".join(review for review in df_1.Clean_Text)
    
    wordlist = text.split()
        
    wordfreq = [wordlist.count(p) for p in wordlist]

    dict = {'Words': wordlist, 'Frequency': wordfreq}

    Dict = dict

    freq_data = pd.DataFrame.from_dict(Dict)
    
    freq_data = freq_data.loc[freq_data['Frequency'] >= min_freq]
    
    freq_data.to_csv("Word_Frequency.csv")
    
    return render_template('frequency.html' , tables = [freq_data.to_html(classes = 'data' , header = 'true')])


@app.route('/wordcloud', methods = ['POST', 'GET'])

def func():
    
    #extract the data from the request
    data = request.get_json()
    doc = data["document"]
    min_freq = data["minimum_frequency"]
    max_words = data["maximum_number_words"]
    Title = data['Title']
  

    path = os.path.dirname(doc)  #this extracts the path for later use. Not sure if it works on apple
    
    df = pd.read_csv(doc)  #read the file
    
    
    #clean the data
    df = df.dropna()
    
    df_1 = df.loc[df['Title'] == str(Title)]
    
    l = list(df_1['reviews_submitted'].unique())
    
    text = " ".join(review for review in df_1.Clean_Text)
    
    WC = WordCloud(background_color="white" , max_font_size = 50,max_words = max_words,  height = 400, width = 700, collocations = False)

    #WC = WordCloud(background_color='white',max_font_size = 50,height=400,width=700, collocations=False)
    
    dic = WC.process_text(text)
    
    test = {}
    for k, v in dic.items():
        if v >= min_freq:
            test.update({k:v})
    
    
    img = WC.fit_words(test)  #buiding the wordcloud with the updated frequency dictinary
    
    plt.imshow(img)
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.show()
    img.to_file("img.png")  #saving the wordcloud to disk
    x = path + "/img.png"   #this is where I used the path again to access the saved wordcloud to send back through flask
    
    return send_file(x, mimetype='image/png')
    
if __name__ == '__main__':
    app.run()  


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


<Figure size 640x480 with 1 Axes>

127.0.0.1 - - [19/Aug/2019 09:40:18] "POST /wordcloud HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2019 09:40:24] "POST /data HTTP/1.1" 200 -
